# 📄 Data Preparation Notebook

**Purpose:** Prepare and convert raw product data (Excel/CSV/JSON) into clean, RAG-ready Markdown for downstream pipelines.

---

## Table of Contents
1. [Setup & Configuration](#setup)
2. [JSON to Markdown](#json2md)
3. [XLSX to CSV (per sheet)](#xlsx2csv)
4. [CSV to Markdown (Q&A)](#csv2md)
5. [Rate Sheet Markdown](#ratesheet)
6. [Product List Markdown](#productlist)
7. [Summary & Next Steps](#summary)

---

**Authors:**
- Ayesh Ahmad (365966)
- Farooq Afzal (365793)
- Muhammad Faras Siddiqui (365988)

**Last Updated:** July 20, 2024

**Version:** 1.1

**Environment:** Python 3.12.10

## 1. Setup & Configuration <a id='setup'></a>
---
This section sets up necessary imports, paths, and configuration for the data processing pipeline.

### 1.1 Imports

In [1]:
# Standard library imports
import os
import json
import re
import logging
from pathlib import Path
from typing import List, Dict, Optional, Union, Tuple, Any
from collections import defaultdict

# Third party imports
import pandas as pd
import numpy as np

# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)
logger = logging.getLogger('data_preparation')

### 1.2 Directory Structure

In [2]:
# Define directory structure
RAW_DIR = Path('./data/raw')
PROC_DIR = Path('./data/processing')
PROCESSED_DIR = Path('./data/processed')
MD_DIR = PROC_DIR / 'mds'
CSV_DIR = PROC_DIR / 'csvs'

# Create directories if they don't exist
for directory in [RAW_DIR, PROC_DIR, PROCESSED_DIR, MD_DIR, CSV_DIR]:
    directory.mkdir(parents=True, exist_ok=True)
    logger.info(f"Directory ensured: {directory}")

2025-05-04 15:43:18 - data_preparation - INFO - Directory ensured: data/raw
2025-05-04 15:43:18 - data_preparation - INFO - Directory ensured: data/processing
2025-05-04 15:43:18 - data_preparation - INFO - Directory ensured: data/processed
2025-05-04 15:43:18 - data_preparation - INFO - Directory ensured: data/processing/mds
2025-05-04 15:43:18 - data_preparation - INFO - Directory ensured: data/processing/csvs


### 1.3 Constants and Configuration

In [3]:
# Configuration Constants
CONFIG = {
    "file_paths": {
        "faq_json": RAW_DIR / 'funds_transfer_app_features_faq.json',
        "products_xlsx": RAW_DIR / 'NUST Bank-Product-Knowledge.xlsx',
        "rate_sheet_csv": CSV_DIR / 'Rate_Sheet_July_1_2024.csv',
    },
    "output_paths": {
        "faq_md": MD_DIR / 'faq.md',
        "rate_sheet_md": MD_DIR / 'Rate_Sheet.md',
        "combined_md": PROC_DIR / 'unreviewed_RAG.md',
    },
    "excluded_csvs": ['Rate_Sheet_July_1_2024.csv', 'Main.csv', 'Sheet1.csv']
}

# List of recognized savings accounts for rate sheet processing
KNOWN_SAVINGS_ACCOUNTS = [
    "NUST Asaan Account",
    "PLS Savings",
    "Little Champs Account", 
    "NUST Special Deposit Account (ASDA)",
    "NUST Waqaar Account - Senior Citizen",
    "PakWatan Remittance Account",
    "NUST Sahar Savings Account",
    "NUST Maximiser Savings Account",
    "PLS Pensioners Account"
]

# Product mapping for final combined markdown
PRODUCTS = {
    'Services and Liability': [
        ('NAA.md', 'NUST Asaan Account'),
        ('CDA.md', 'Current Deposit Account'),
        ('VPCA.md', 'Value Plus Current Account'),
        ('VP-BA.md', 'Value Plus Business Account'),
        ('VPBA.md', 'Value Premium Business Account'),
        ('NSDA.md', 'NUST Special Deposit Account'),
        ('PLS.md', 'Profit And Loss Sharing Account'),
        ('NFDA.md', 'NUST Freelancer Deposit Account'),
        ('NMA.md', 'NUST Maximiser Account'),
        ('NADRA.md', 'NUST Asaan Digital Remittance Account'),
        ('NSA.md', 'NUST Sahar Account'),
        ('NADA.md', 'NUST Asaan Digital Account'),
        ('PWRA.md', 'PakWatan Remittance Account'),
        ('RDA.md', 'Roshan Digital Account'),
        ('LCA.md', 'Little Champs Account'),
        ('NWA.md', 'NUST Waqaar Account'),
        ('HOME_REMITTANCE.md', 'NUST Home Remittance'),
    ],
    'Consumer': [
        ('NMC.md', 'NUST MasterCard'),
        ('NUST4Car.md', 'NUST4Car'),
        ('NMF.md', 'NUST Mortgage Finance'),
        ('PF.md', 'Personal Finance'),
    ],
    'SME': [
        ('NUF.md', 'NUST Ujala Finance'),
        ('NSF.md', 'NUST Sahar Finance'),
        ('NFBF.md', 'NUST Fauri Business Finance'),
        ('PMYB__ALS.md', 'Prime Minister Youth Business and Agriculture Loan Scheme'),
        ('NHF.md', 'NUST Hunarmand Finance'),
        ('NFMF.md', 'NUST Flour Mill Finance'),
        ('NIF.md', 'NUST Imarat Finance'),
        ('NRF.md', 'NUST Rice Finance'),
    ],
    'Third Party': [
        ('NUST_Life.md', 'NUST Life Bancassurance Policy'),
        ('EFU_Life.md', 'EFU Life Bancassurance Policy'),
        ('Jubilee_Life_.md', 'Jubilee Life Bancassurance Policy'),
    ],
    'Other': [
        ('ESFCA.md', 'Exporters Special Foreign Currency Account'),
        ('Rate_Sheet.md', 'Rate Sheet'),
    ]
}

### 1.4 Utility Functions

In [4]:
def save_markdown(markdown_text: str, output_path: Path) -> None:
    """Save Markdown content to a file.
    
    Args:
        markdown_text: The markdown content to save
        output_path: Path to save the markdown file
    """
    output_path.parent.mkdir(parents=True, exist_ok=True)
    
    with open(output_path, 'w', encoding='utf-8') as f:
        f.write(markdown_text)
    
    logger.info(f"Saved markdown file: {output_path}")

def read_file(file_path: Path) -> str:
    """Read file content if it exists.
    
    Args:
        file_path: Path to the file to read
        
    Returns:
        str: File content as string, empty string if file not found
    """
    if file_path.exists():
        with open(file_path, 'r', encoding='utf-8') as f:
            return f.read()
    
    logger.warning(f"File not found: {file_path.name}")
    return ''

## 2. JSON to Markdown <a id='json2md'></a>
---
This section handles the conversion of structured FAQ JSON data to a more readable and RAG-friendly Markdown format.

### 2.1 JSON Processing Functions

In [5]:
def load_json(file_path: Path) -> dict:
    """Load JSON data from a file.
    
    Args:
        file_path: Path to the JSON file
        
    Returns:
        dict: Loaded JSON data as dictionary
        
    Raises:
        FileNotFoundError: If the file doesn't exist
        json.JSONDecodeError: If the file contains invalid JSON
    """
    logger.info(f"Loading JSON from: {file_path}")
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            return json.load(f)
    except FileNotFoundError:
        logger.error(f"JSON file not found: {file_path}")
        raise
    except json.JSONDecodeError as e:
        logger.error(f"Invalid JSON format in {file_path}: {str(e)}")
        raise

def json_to_faq_markdown(data: dict) -> str:
    """Convert structured FAQ JSON to RAG-friendly Markdown.
    
    Expected JSON structure:
    {
        "categories": [
            {
                "category": "Category Name",
                "questions": [
                    {"question": "Question text?", "answer": "Answer text"}
                ]
            }
        ]
    }
    
    Args:
        data: JSON data as a dictionary
        
    Returns:
        str: Markdown formatted FAQ content
    """
    logger.info("Converting JSON data to FAQ markdown")
    markdown = []
    
    # Track statistics for logging
    category_count = 0
    question_count = 0
    
    for category in data.get('categories', []):
        cat_title = category.get('category', 'General')
        category_count += 1
        
        markdown.append(f'\n## FAQ: {cat_title}\n')
        
        for qa in category.get('questions', []):
            question = qa.get('question', '').strip()
            answer = qa.get('answer', '').strip()
            
            if question and answer:
                markdown.append(f'\n#### Q: {question}\n**A:**  \n{answer}\n')
                question_count += 1
    
    logger.info(f"Converted {category_count} categories with {question_count} Q&A pairs")
    return '\n'.join(markdown).strip()

### 2.2 Process FAQ JSON to Markdown

In [6]:
def process_faq_json():
    """Process FAQ JSON file to markdown.
    
    Uses the configuration defined in CONFIG dictionary.
    """
    # Get file paths from config
    json_input_path = CONFIG["file_paths"]["faq_json"]
    json_output_path = CONFIG["output_paths"]["faq_md"]
    
    logger.info(f"Processing FAQ JSON: {json_input_path} → {json_output_path}")
    
    try:
        # Load and process JSON data
        json_data = load_json(json_input_path)
        md_text = json_to_faq_markdown(json_data)
        
        # Save the resulting markdown
        save_markdown(md_text, json_output_path)
        logger.info(f"Successfully converted FAQ JSON to markdown")
        
        return True
    except Exception as e:
        logger.error(f"Error processing FAQ JSON: {str(e)}")
        return False

In [7]:
# Run the FAQ JSON conversion
process_faq_json()
print(f'[✓] Saved Markdown from JSON: {CONFIG["output_paths"]["faq_md"]}')

2025-05-04 15:44:44 - data_preparation - INFO - Processing FAQ JSON: data/raw/funds_transfer_app_features_faq.json → data/processing/mds/faq.md
2025-05-04 15:44:44 - data_preparation - INFO - Loading JSON from: data/raw/funds_transfer_app_features_faq.json
2025-05-04 15:44:44 - data_preparation - INFO - Converting JSON data to FAQ markdown
2025-05-04 15:44:44 - data_preparation - INFO - Converted 2 categories with 15 Q&A pairs
2025-05-04 15:44:44 - data_preparation - INFO - Saved markdown file: data/processing/mds/faq.md
2025-05-04 15:44:44 - data_preparation - INFO - Successfully converted FAQ JSON to markdown


[✓] Saved Markdown from JSON: data/processing/mds/faq.md


## 3. XLSX to CSV (per sheet) <a id='xlsx2csv'></a>
---
This section handles the conversion of Excel workbook sheets into individual CSV files for further processing.

### 3.1 XLSX to CSV Functions

In [8]:
def convert_xlsx_to_csvs(xlsx_file_path: Path, output_dir: Path = None) -> List[Path]:
    """Convert each sheet in an XLSX file to individual CSVs.
    
    Args:
        xlsx_file_path: Path to the Excel file
        output_dir: Directory to save CSV files in (default: dir with file name)
        
    Returns:
        List[Path]: List of paths to the generated CSV files
        
    Raises:
        FileNotFoundError: If the Excel file doesn't exist
        ValueError: If the Excel file is invalid or empty
    """
    logger.info(f"Converting Excel file to CSVs: {xlsx_file_path}")
    
    if not xlsx_file_path.exists():
        logger.error(f"Excel file not found: {xlsx_file_path}")
        raise FileNotFoundError(f"Excel file not found: {xlsx_file_path}")
    
    # Set up output directory
    output_dir = output_dir or xlsx_file_path.with_suffix('')
    output_dir.mkdir(parents=True, exist_ok=True)
    
    # Open Excel file
    try:
        excel_file = pd.ExcelFile(xlsx_file_path)
        sheet_names = excel_file.sheet_names
        
        if not sheet_names:
            logger.warning(f"No sheets found in Excel file: {xlsx_file_path}")
            return []
            
        logger.info(f"Found {len(sheet_names)} sheets in {xlsx_file_path.name}")
    except Exception as e:
        logger.error(f"Error opening Excel file: {str(e)}")
        raise ValueError(f"Invalid Excel file: {str(e)}")
    
    # Process each sheet
    csv_files = []
    
    for sheet_name in sheet_names:
        try:
            df = excel_file.parse(sheet_name)
            
            # Create a safe filename from sheet name
            safe_name = re.sub(r'[\s/&]', '_', sheet_name)
            csv_file_path = output_dir / f'{safe_name}.csv'
            
            # Save as CSV
            df.to_csv(csv_file_path, index=False)
            csv_files.append(csv_file_path)
            
            logger.info(f"Exported sheet '{sheet_name}' to {csv_file_path}")
        except Exception as e:
            logger.error(f"Error processing sheet '{sheet_name}': {str(e)}")
    
    return csv_files

### 3.2 Process Excel to CSVs

In [9]:
def process_xlsx_to_csvs():
    """Process XLSX file to CSV files.
    
    Uses the configuration defined in CONFIG dictionary.
    """
    # Get file paths from config
    xlsx_input = CONFIG["file_paths"]["products_xlsx"]
    
    try:
        # Convert XLSX to CSVs
        csv_files = convert_xlsx_to_csvs(xlsx_input, CSV_DIR)
        logger.info(f"Successfully converted {len(csv_files)} sheets to CSV format")
        return len(csv_files)
    except Exception as e:
        logger.error(f"Error processing Excel file: {str(e)}")
        return 0

# Run the Excel to CSV conversion
num_csvs = process_xlsx_to_csvs()
print(f'[✓] Exported {num_csvs} sheets to CSV format')

2025-05-04 15:45:13 - data_preparation - INFO - Converting Excel file to CSVs: data/raw/NUST Bank-Product-Knowledge.xlsx
2025-05-04 15:45:15 - data_preparation - INFO - Found 36 sheets in NUST Bank-Product-Knowledge.xlsx
2025-05-04 15:45:15 - data_preparation - INFO - Exported sheet 'Main' to data/processing/csvs/Main.csv
2025-05-04 15:45:15 - data_preparation - INFO - Exported sheet 'Rate Sheet July 1 2024' to data/processing/csvs/Rate_Sheet_July_1_2024.csv
2025-05-04 15:45:15 - data_preparation - INFO - Exported sheet 'LCA' to data/processing/csvs/LCA.csv
2025-05-04 15:45:15 - data_preparation - INFO - Exported sheet 'NAA' to data/processing/csvs/NAA.csv
2025-05-04 15:45:15 - data_preparation - INFO - Exported sheet 'NWA' to data/processing/csvs/NWA.csv
2025-05-04 15:45:15 - data_preparation - INFO - Exported sheet 'PWRA' to data/processing/csvs/PWRA.csv
2025-05-04 15:45:15 - data_preparation - INFO - Exported sheet 'RDA' to data/processing/csvs/RDA.csv
2025-05-04 15:45:15 - data_pre

[✓] Exported 36 sheets to CSV format


## 4. CSV to Markdown (Q&A) <a id='csv2md'></a>
---
This section handles the conversion of Q&A CSVs to structured markdown documents using custom rules and heuristics.

### 4.1 Q&A Text Processing Functions

In [10]:
def is_question(text: str) -> bool:
    """Determine if a given text is likely a question.
    
    Uses heuristics like question marks and question-starting words.
    
    Args:
        text: The text to evaluate
        
    Returns:
        bool: True if text is likely a question
    """
    if not isinstance(text, str):
        return False
        
    question_words = (
        'what', 'how', 'why', 'when', 'where', 'who', 'which', 
        'can', 'is', 'are', 'do', 'does', 'will', 'would', 'should', 'could'
    )
    text_lower = text.lower().strip()
    
    return '?' in text or any(text_lower.startswith(word) for word in question_words)

def merge_short_value_to_previous(lines: list) -> list:
    """Merge short numeric/text values with previous lines.
    
    Helps with formatting answers where values like rates or numbers
    are split onto their own lines but should be part of previous text.
    
    Args:
        lines: List of text lines to process
        
    Returns:
        list: List with short values merged with previous lines
    """
    merged = []
    
    for line in lines:
        # If line is short, alphanumeric, and we have previous content
        if (len(str(line).split()) <= 2 and 
            re.match(r'^[\d.,a-zA-Z\s]+$', str(line)) and 
            merged):
            merged[-1] += f' {line}'
        else:
            merged.append(str(line))
            
    return merged

def format_answer(lines: list) -> str:
    """Format a list of answer lines into markdown.
    
    Handles bullet points, formatting, and merges short values.
    
    Args:
        lines: List of answer text lines
        
    Returns:
        str: Formatted markdown for the answer
    """
    # Merge short numeric values with previous lines
    lines = merge_short_value_to_previous(lines)
    
    formatted_lines = []
    for line in lines:
        clean = str(line).lstrip('o ').strip()
        
        # Handle bullet points
        if (line.startswith(('o ', 'o', '*', '-')) or 
            clean.lower().startswith('free') or 
            clean.endswith(':')):
            formatted_lines.append(f'* {clean}')
        else:
            formatted_lines.append(clean)
            
    return '\n'.join(formatted_lines)

### 4.2 CSV Content Extraction Functions

In [11]:
def extract_blocks(df: pd.DataFrame) -> list:
    """Extract Q&A blocks from a dataframe.
    
    Scans through the dataframe to identify question-answer pairs
    based on heuristics like question structure.
    
    Args:
        df: DataFrame containing Q&A content
        
    Returns:
        list: List of (question, answer_lines) tuples
    """
    # Drop completely empty rows
    df = df.dropna(how='all')
    blocks = []
    
    current_question = None
    current_answer = []
    
    for _, row in df.iterrows():
        # Get non-empty cells from the row
        cells = [str(cell).strip() for cell in row if pd.notna(cell) and str(cell).strip()]
        
        for line in cells:
            if is_question(line):
                # If we have a previous question, add it to blocks
                if current_question:
                    blocks.append((current_question, current_answer))
                    
                # Start new question
                current_question = line
                current_answer = []
            elif current_question:
                # Add line to current answer
                current_answer.append(line)
    
    # Add the last question if any
    if current_question:
        blocks.append((current_question, current_answer))
        
    return blocks

def convert_blocks_to_markdown(blocks: list) -> str:
    """Convert Q&A blocks to markdown format.
    
    Args:
        blocks: List of (question, answer_lines) tuples
        
    Returns:
        str: Markdown formatted Q&A content
    """
    markdown_blocks = []
    
    for question, answer_lines in blocks:
        markdown = f'#### Q: {question.strip()}\n**A:**\n'
        formatted_answer = format_answer(answer_lines)
        markdown += formatted_answer
        markdown_blocks.append(markdown)
        
    return '\n\n'.join(markdown_blocks)

### 4.3 CSV to Markdown Conversion Functions

In [12]:
def process_csv_to_markdown(csv_path: Path, output_dir: Path = None) -> Optional[Path]:
    """Process a CSV file to Markdown Q&A format.
    
    Args:
        csv_path: Path to the CSV file
        output_dir: Directory to save markdown file (default: MD_DIR)
        
    Returns:
        Optional[Path]: Path to the generated markdown file, or None if failed
    """
    output_dir = output_dir or MD_DIR
    logger.info(f"Processing {csv_path} to markdown")
    
    try:
        # Read CSV without headers
        df = pd.read_csv(csv_path, header=None)
        
        # Extract Q&A blocks
        blocks = extract_blocks(df)
        if not blocks:
            logger.warning(f"No content blocks found in {csv_path}")
            return None
            
        # Generate markdown
        markdown_text = convert_blocks_to_markdown(blocks)
        
        # Save markdown file
        base_name = csv_path.stem
        output_path = output_dir / f'{base_name}.md'
        save_markdown(markdown_text, output_path)
        
        logger.info(f"Processed {csv_path.name} → {output_path.name} with {len(blocks)} Q&A blocks")
        return output_path
        
    except pd.errors.EmptyDataError:
        logger.warning(f"Empty or invalid CSV file: {csv_path}")
        return None
    except Exception as e:
        logger.error(f"Error processing {csv_path}: {str(e)}")
        return None

### 4.4 Batch Process CSVs to Markdown

In [13]:
def batch_process_csvs_to_markdown() -> Tuple[int, int]:
    """Batch process all CSV files to markdown (excluding specified ones).
    
    Returns:
        Tuple[int, int]: (number of successful conversions, number of failures)
    """
    logger.info(f"Starting batch conversion of CSVs to markdown")
    
    # Get list of all CSV files
    excluded_files = set(CONFIG["excluded_csvs"])
    csv_files = [f for f in os.listdir(CSV_DIR) if f.endswith('.csv') and f not in excluded_files]
    
    logger.info(f"Found {len(csv_files)} CSV files to process (excluding {len(excluded_files)} files)")
    
    # Process each CSV file
    success_count = 0
    error_count = 0
    
    for csv_file in csv_files:
        csv_path = CSV_DIR / csv_file
        result = process_csv_to_markdown(csv_path)
        
        if result:
            success_count += 1
        else:
            error_count += 1
            
    logger.info(f"CSV processing complete: {success_count} successes, {error_count} errors")
    return success_count, error_count

# Run the batch CSV to markdown conversion
success_count, error_count = batch_process_csvs_to_markdown()
print(f'[✓] Processed {success_count} CSV files to markdown format')
if error_count > 0:
    print(f'[!] {error_count} files had processing errors')

2025-05-04 15:46:02 - data_preparation - INFO - Starting batch conversion of CSVs to markdown
2025-05-04 15:46:02 - data_preparation - INFO - Found 33 CSV files to process (excluding 3 files)
2025-05-04 15:46:02 - data_preparation - INFO - Processing data/processing/csvs/NWA.csv to markdown
2025-05-04 15:46:02 - data_preparation - INFO - Saved markdown file: data/processing/mds/NWA.md
2025-05-04 15:46:02 - data_preparation - INFO - Processed NWA.csv → NWA.md with 5 Q&A blocks
2025-05-04 15:46:02 - data_preparation - INFO - Processing data/processing/csvs/Jubilee_Life_.csv to markdown
2025-05-04 15:46:02 - data_preparation - INFO - Saved markdown file: data/processing/mds/Jubilee_Life_.md
2025-05-04 15:46:02 - data_preparation - INFO - Processed Jubilee_Life_.csv → Jubilee_Life_.md with 4 Q&A blocks
2025-05-04 15:46:02 - data_preparation - INFO - Processing data/processing/csvs/VPBA.csv to markdown
2025-05-04 15:46:02 - data_preparation - INFO - Saved markdown file: data/processing/mds/

[✓] Processed 33 CSV files to markdown format


## 5. Rate Sheet Markdown <a id='ratesheet'></a>
---
This section handles the extraction and formatting of rate sheet data into structured markdown tables.

### 5.1 Rate Sheet Extraction Utilities

In [14]:
def extract_declaration(df: pd.DataFrame) -> str:
    """Extract declaration text from first few rows of CSV.
    
    Args:
        df: DataFrame containing the rate sheet data
        
    Returns:
        str: Declaration text or empty string if not found
    """
    for i in range(min(5, len(df))):
        cell = str(df.iloc[i, 1]).strip() if pd.notna(df.iloc[i, 1]) else ""
        if "Indicative Profit Rates" in cell:
            return cell.replace("\n", " ").strip('"')
    return ""

def is_savings_account(cell: str) -> bool:
    """Check if cell contains a savings account name.
    
    Args:
        cell: Cell content to check
        
    Returns:
        bool: True if cell likely contains a savings account name
    """
    if not isinstance(cell, str):
        return False
        
    cell = cell.strip().replace(",", "")
    
    if not cell or cell in ["Profit Payment", "Profit Rate", "Tenor"]:
        return False
        
    return cell in KNOWN_SAVINGS_ACCOUNTS or cell.endswith("Account")

def tenor_sorter(tenor: str) -> int:
    """Sort tenors by duration for consistent order in markdown.
    
    Args:
        tenor: Tenor string like "3 Months" or "1 Year"
        
    Returns:
        int: Numeric value (in days) for sorting
    """
    # Extract numeric value if present
    num = re.search(r'(\d+)', tenor)
    if num:
        value = int(num.group(1))
        return {
            "Day": value,
            "Month": value * 30,
            "Year": value * 365
        }.get(next((unit for unit in ["Day", "Month", "Year"] if unit in tenor), ""), 9999)
    
    # Handle text-based tenors
    for unit, prefixes in {
        "Month": {"One": 30, "Two": 60, "Three": 90, "Six": 180},
        "Year": {"One": 365, "Two": 730, "Three": 1095, "Five": 1825}
    }.items():
        if unit in tenor:
            return next((days for prefix, days in prefixes.items() if prefix in tenor), 9999)
    
    return 9999

### 5.2 Rate Sheet Data Extraction Functions

In [15]:
def extract_savings_accounts(df: pd.DataFrame) -> pd.DataFrame:
    """Extract savings account information from dataframe.
    
    Args:
        df: DataFrame containing the rate sheet data
        
    Returns:
        pd.DataFrame: DataFrame with savings account data
    """
    logger.info("Extracting savings account information")
    rows = []
    current_account = None

    for i, row in df.iterrows():
        # Look for account names
        cell = str(row[1]).strip() if pd.notna(row[1]) else ""
        if is_savings_account(cell):
            current_account = cell
            
        # Look for profit payment rows
        if isinstance(row[1], str) and "Profit Payment" in row[1]:
            # Check next few rows for payment frequencies
            for j in range(1, 5):
                if i + j >= len(df):
                    break
                    
                freq = df.iloc[i + j, 1]
                
                # Find rate in columns to the right
                rate = next(
                    (df.iloc[i + j, k] for k in range(2, 6) 
                     if k < len(df.columns) and pd.notna(df.iloc[i + j, k]) 
                     and re.match(r"^\d*\.?\d+$", str(df.iloc[i + j, k]).strip())), 
                    None
                )
                
                # Check if we have valid payment frequency
                if (isinstance(freq, str) and 
                    re.match(r"^(Monthly|Quarterly|Semi-Annually|Annually)", freq.strip()) and 
                    current_account):
                    rows.append({
                        "Account Name": current_account,
                        "Profit Payment": freq.strip(),
                        "Profit Rate": float(rate) if rate else None,
                    })
    
    logger.info(f"Found {len(rows)} savings accounts")
    return pd.DataFrame(rows)

def process_bachat_account(df: pd.DataFrame, i: int) -> List[Dict]:
    """Process NUST Bachat Account entries.
    
    Args:
        df: DataFrame containing the rate sheet data
        i: Row index to start from
        
    Returns:
        List[Dict]: List of Bachat Account entry dictionaries
    """
    rows = []
    j = i + 2
    
    while j < len(df) and j < i + 5:
        next_row = df.iloc[j].fillna("").astype(str).tolist()
        if len(next_row) > 8:
            tenor = next_row[5].strip()
            payout = next_row[6].strip()
            rate = next_row[8].strip()
            
            if all([tenor, payout, rate]) and re.match(r"^\d*\.?\d+$", rate):
                rows.append({
                    "Account Name": "NUST Bachat Account",
                    "Tenor": tenor,
                    "Payout": payout,
                    "Profit Rate": float(rate),
                })
        j += 1
        
    return rows

def get_account_name(df: pd.DataFrame, i: int, default_name: str) -> str:
    """Get account name for term deposits.
    
    Args:
        df: DataFrame containing the rate sheet data
        i: Row index
        default_name: Default name to use if better one isn't found
        
    Returns:
        str: Account name
    """
    account_name = default_name.strip()
    if "Deposit Receipt" in default_name or "Term Deposit" in default_name:
        return account_name
    
    account_name = "Regular Term Deposit"  # Default fallback
    
    # Check previous row for better name
    if i > 0:
        prev_row = df.iloc[i-1].fillna("").astype(str).tolist()
        if len(prev_row) > 5 and prev_row[5].strip():
            account_name = prev_row[5].strip()
            
    return account_name

def process_term_deposit_rows(df: pd.DataFrame, i: int) -> List[Dict]:
    """Process term deposit rows.
    
    Args:
        df: DataFrame containing the rate sheet data
        i: Row index to start from
        
    Returns:
        List[Dict]: List of term deposit entry dictionaries
    """
    rows = []
    j = i + 2
    
    while j < len(df):
        next_row = df.iloc[j].fillna("").astype(str).tolist()
        if len(next_row) <= 8:
            j += 1
            continue
            
        tenor = next_row[5].strip()
        payout = next_row[6].strip()
        rate = next_row[8].strip()
        
        # Check if we've reached the end of the deposit section
        if not tenor or (not re.search(r"\d", tenor) and 
                        not any(word in tenor for word in ["One", "Two", "Three", "Six", "Five"])):
            break
            
        # Process valid rate entries
        if re.match(r"^\d*\.?\d+$", rate):
            rows.append({
                "Account Name": get_account_name(df, i, df.iloc[i][5]),
                "Tenor": tenor,
                "Payout": payout if payout else "Maturity",
                "Profit Rate": float(rate),
            })
            
        j += 1
        
    return rows

def extract_term_deposits(df: pd.DataFrame) -> pd.DataFrame:
    """Extract term deposit information from dataframe.
    
    Args:
        df: DataFrame containing the rate sheet data
        
    Returns:
        pd.DataFrame: DataFrame with term deposit data
    """
    logger.info("Extracting term deposit information")
    rows = []

    for i in range(len(df)):
        row = df.iloc[i].fillna("").astype(str).tolist()
        
        if len(row) <= 5 or row[5] == "":
            continue
            
        # Process NUST Bachat Account
        if "NUST Bachat Account" in row[5]:
            rows.extend(process_bachat_account(df, i))
            continue

        # Process Term Deposits
        if "Term Deposits" in row[5] or any(term in row[5] for term in ["Deposit Receipt", "Term Deposit"]):
            if i + 1 >= len(df):
                continue
                
            next_row = df.iloc[i+1].fillna("").astype(str).tolist()
            if len(next_row) > 5 and "Tenor" in next_row[5]:
                account_name = get_account_name(df, i, row[5])
                
                # Skip headers or empty sections
                if any(phrase in account_name for phrase in ["Change", "TERM DEPOSITS"]):
                    continue
                
                rows.extend(process_term_deposit_rows(df, i))

    logger.info(f"Found {len(rows)} term deposits")
    return pd.DataFrame(rows)

def extract_fcy_rates(df: pd.DataFrame) -> pd.DataFrame:
    """Extract foreign currency rates from dataframe.
    
    Args:
        df: DataFrame containing the rate sheet data
        
    Returns:
        pd.DataFrame: DataFrame with FCY rate data
    """
    logger.info("Extracting foreign currency (FCY) rates")
    fcy_data = []
    
    for i in range(len(df)):
        row = df.iloc[i].fillna("").astype(str).tolist()
        if len(row) > 6 and "FCY" in row[5]:
            for j in range(i+1, i+4):
                if j < len(df):
                    next_row = df.iloc[j].fillna("").astype(str).tolist()
                    if len(next_row) > 8:
                        account_type = next_row[5].strip()
                        if not account_type:
                            continue
                            
                        usd, gbp, eur = map(str.strip, next_row[6:9])
                        
                        if re.match(r"^\d*\.?\d+$", usd):
                            fcy_data.append({
                                "Account Type": account_type,
                                "USD": float(usd) * 100,  # Convert to percentage
                                "GBP": float(gbp) * 100,
                                "EUR": float(eur) * 100
                            })
    
    logger.info(f"Found {len(fcy_data)} FCY rates")
    return pd.DataFrame(fcy_data)

### 5.3 Rate Sheet Markdown Generation

In [16]:
def generate_rate_sheet_markdown(csv_path: Path, output_path: Path = None) -> Optional[Path]:
    """Generate markdown from CSV rate sheet.
    
    Args:
        csv_path: Path to rate sheet CSV
        output_path: Path to save the markdown file (default: from CONFIG)
        
    Returns:
        Optional[Path]: Path to the generated markdown file, or None if failed
    """
    output_path = output_path or CONFIG["output_paths"]["rate_sheet_md"]
    logger.info(f"Processing rate sheet from {csv_path}")
    
    try:
        # Read CSV file
        df = pd.read_csv(csv_path, header=None)
        
        # Extract data sections
        declaration = extract_declaration(df)
        logger.info("Extracting savings accounts...")
        savings_df = extract_savings_accounts(df)
        
        logger.info("Extracting term deposits...")
        term_deposits_df = extract_term_deposits(df)
        
        logger.info("Extracting FCY rates...")
        fcy_df = extract_fcy_rates(df)
        
        # Sort term deposits by tenor
        if not term_deposits_df.empty:
            term_deposits_df['SortOrder'] = term_deposits_df['Tenor'].apply(tenor_sorter)
            term_deposits_df = term_deposits_df.sort_values(['Account Name', 'SortOrder'])
            term_deposits_df = term_deposits_df.drop('SortOrder', axis=1)
        
        # Generate markdown
        logger.info("Generating markdown output...")
        with open(output_path, "w", encoding="utf-8") as f:
            # Add disclaimer/declaration if available
            if declaration:
                f.write(f"*{declaration}*\n\n")
            
            # Add savings accounts section
            f.write("## Savings Accounts\n\n")
            f.write(savings_df.to_markdown(index=False))
            f.write("\n\n")
            
            # Add term deposits section
            f.write("## Term Deposit Accounts\n\n")
            f.write(term_deposits_df.to_markdown(index=False))
            f.write("\n\n")
            
            # Add FCY rates section if available
            if not fcy_df.empty:
                f.write("## Foreign Currency (FCY) Rates\n\n")
                fcy_df_formatted = fcy_df.copy()
                for col in ['USD', 'GBP', 'EUR']:
                    if col in fcy_df.columns:
                        fcy_df_formatted[col] = fcy_df_formatted[col].apply(
                            lambda x: f"{x}%" if isinstance(x, (int, float)) else x
                        )
                f.write(fcy_df_formatted.to_markdown(index=False))
                f.write("\n\n")
        
        logger.info(f"Rate sheet markdown saved to {output_path}")
        return output_path
        
    except Exception as e:
        logger.error(f"Error processing rate sheet {csv_path}: {str(e)}")
        return None

# Process rate sheet to markdown
rate_sheet_path = generate_rate_sheet_markdown(CONFIG["file_paths"]["rate_sheet_csv"])
if rate_sheet_path:
    print(f'[✓] Rate sheet markdown generated successfully')
else:
    print(f'[!] Failed to generate rate sheet markdown')

2025-05-04 15:47:12 - data_preparation - INFO - Processing rate sheet from data/processing/csvs/Rate_Sheet_July_1_2024.csv
2025-05-04 15:47:12 - data_preparation - INFO - Extracting savings accounts...
2025-05-04 15:47:12 - data_preparation - INFO - Extracting savings account information
2025-05-04 15:47:12 - data_preparation - INFO - Found 12 savings accounts
2025-05-04 15:47:12 - data_preparation - INFO - Extracting term deposits...
2025-05-04 15:47:12 - data_preparation - INFO - Extracting term deposit information
2025-05-04 15:47:12 - data_preparation - INFO - Found 23 term deposits
2025-05-04 15:47:12 - data_preparation - INFO - Extracting FCY rates...
2025-05-04 15:47:12 - data_preparation - INFO - Extracting foreign currency (FCY) rates
2025-05-04 15:47:12 - data_preparation - INFO - Found 2 FCY rates
2025-05-04 15:47:12 - data_preparation - INFO - Generating markdown output...
2025-05-04 15:47:12 - data_preparation - INFO - Rate sheet markdown saved to data/processing/mds/Rate_

[✓] Rate sheet markdown generated successfully


## 6. Product List Markdown <a id='productlist'></a>
---
This section combines all the generated markdown files into a single comprehensive document.

### 6.1 Markdown Combination Function

In [17]:
def merge_markdown_files(output_path: Path = None) -> Optional[Path]:
    """Merge all markdown files into a single comprehensive document.
    
    Args:
        output_path: Path to save the combined markdown (default: from CONFIG)
        
    Returns:
        Optional[Path]: Path to the combined markdown file, or None if failed
    """
    output_path = output_path or CONFIG["output_paths"]["combined_md"]
    dir_path = MD_DIR
    
    logger.info(f"Merging markdown files from {dir_path} to {output_path}")
    
    try:
        # Ensure output directory exists
        output_path.parent.mkdir(parents=True, exist_ok=True)
        
        with open(output_path, 'w', encoding='utf-8') as outfile:
            # Write overview section
            outfile.write("# NUST Bank Products Overview\n\n")
            
            # Write product categories in table of contents
            for category, files in PRODUCTS.items():
                outfile.write(f"## {category} Products\n\n")
                for _, new_name in files:
                    outfile.write(f"- {new_name.replace('.md', '')}\n")
                outfile.write("\n")

            # Write FAQ section first
            outfile.write("\n# FAQ\n\n")
            outfile.write(read_file(dir_path / 'faq.md'))
            outfile.write("\n\n")

            # Write Rate Sheet section second
            outfile.write("\n# Rate Sheet\n\n")
            outfile.write(read_file(dir_path / 'Rate_Sheet.md'))
            outfile.write("\n\n")

            # Write detailed product sections
            outfile.write("\n# Detailed Product Information\n\n")
            
            # Process each category
            product_count = 0
            for category, files in PRODUCTS.items():
                outfile.write(f"## {category} Products\n\n")
                
                for old_name, new_name in files:
                    # Skip rate sheet as it's already included
                    if old_name != 'Rate_Sheet.md':
                        outfile.write(f"### {new_name.replace('.md', '')}\n\n")
                        content = read_file(dir_path / old_name)
                        outfile.write(content)
                        outfile.write("\n\n")
                        product_count += 1
        
        logger.info(f"Combined markdown with {product_count} products saved to {output_path}")
        return output_path
        
    except Exception as e:
        logger.error(f"Error merging markdown files: {str(e)}")
        return None

# Merge all markdown files
combined_path = merge_markdown_files()
if combined_path:
    print(f'[✓] Combined markdown document generated: {combined_path}')
else:
    print(f'[!] Failed to generate combined markdown document')

2025-05-04 15:47:54 - data_preparation - INFO - Merging markdown files from data/processing/mds to data/processing/unreviewed_RAG.md
2025-05-04 15:47:54 - data_preparation - INFO - Combined markdown with 33 products saved to data/processing/unreviewed_RAG.md


[✓] Combined markdown document generated: data/processing/unreviewed_RAG.md


## 7. Summary & Next Steps <a id='summary'></a>
---
This section summarizes the data processing pipeline and provides guidance for manual review and next steps.

### 7.1 Manual Review Notes

The generated markdown document (`unreviewed_RAG.md`) requires manual review for optimal formatting. Manually reviewing our dataset makes it high-value, more accurate, and trustworthy as opposed to using a LLM.

Here are the specific items to review:
 
1. **Extra Bullet Points**: Some markdown files may contain redundant bullet points, particularly when the original CSV had ambiguous list formatting.
 
2. **Inconsistent Headers**: Check for proper heading level hierarchy throughout the document.
 
3. **Table Formatting**: Ensure all tables in the Rate Sheet section are properly aligned and readable.
 
4. **Image References**: If any markdown refers to images, ensure the paths are correct or the references are removed.
 
5. **Duplicate Content**: Some products may have overlapping information that should be consolidated.
 
After manual review and editing, save the final version to `data/processed/RAG.md` for use in downstream pipelines.
 
📝 NEXT STEPS:
1. Manually review the 'data/processing/unreviewed_RAG.md' file
2. Fix any formatting issues (e.g., extra bullets, inconsistent headers)
3. Save final version to 'data/processed/RAG.md'
4. Use the final document for downstream RAG processing

⚠️ KNOWN FORMATTING ISSUES TO CHECK:
- Extra bullet points
- Redundant bullets within bullets
- Bullets using "·" instead of "*"
- Inconsistent spacing between sections
- Table formattings
- Missing or malformed Q/A pairs
- Junk instances of "Main" in the markdown
